In [ ]:
import os
import re
import ast
import json
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
import spacy

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
df = pd.read_csv("../input/roberta-config/save.csv")
df_tr = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
def string_to_list_loc(location_list):
    output = []
    location_list = ast.literal_eval(location_list)
    for location in location_list:
        for loc in [s.split() for s in location.split(';')]:
            output.append((int(loc[0]), int(loc[1])))
    return output

df["span"] = df["span"].apply(lambda x: ast.literal_eval(x))
df["location"] = df_tr["location"].apply(lambda x: string_to_list_loc(x))

In [ ]:
def micro_f1(preds, truths):
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds)

def spans_to_binary(spans, length=None):
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary

def span_micro_f1(preds, truths):
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

In [ ]:
err_idx = {"empty_pre": [],
          "empty_true": [],
          "low": [],
          "medium": [],
          "high": []}
for i in range(len(df)):
    pred = df["span"].iloc[i]
    true = df["location"].iloc[i]
    if len(pred) == 0 and len(true) == 0:
        continue
    if len(pred) == 0:
        err_idx["empty_pre"].append(i)
        continue
    if len(true) == 0:
        err_idx["empty_true"].append(i)
        continue
    f1 = span_micro_f1([pred], [true])
    if f1 == 1.0:
        continue
    if 0.7 <= f1 < 1.0:
        err_idx["high"].append((i,f1))
    elif 0.5 <= f1 < 0.7:
        err_idx["medium"].append((i,f1))
    else:
        err_idx["low"].append((i,f1))
for key in err_idx.keys():
    print(key, len(err_idx[key]))

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [ ]:
def print_error(key, num):
    string = f"There total {len(err_idx[key])} {key}-error examples"
    print(bcolors.WARNING + bcolors.BOLD + string + bcolors.ENDC)
    
    for i in range(10):
        idx, f1 = err_idx[key][i]
        string = "Feature text: " + df["feature_text"].iloc[idx]
        print(bcolors.BOLD + bcolors.OKGREEN + string + bcolors.ENDC)
     
        string = "="*20 + f"Error: {key} | TRUE LABEL | Example: {i+1}/{num} | F1: {f1}" + "="*20
        print(bcolors.BOLD + string + bcolors.ENDC)
        plot_annotation_by_idx(df, idx, choice="span")
        
        string = "="*20 + f"Error: {key} | MODEL PREDICT | Example: {i+1}/{num} | F1: {f1}" + "="*20
        print(bcolors.BOLD + string + bcolors.ENDC)
        plot_annotation_by_idx(df, idx, choice="location")
        
        print("\n")

In [ ]:
def plot_annotation_by_idx(df, idx, choice="span"):
    options = {"colors": {}}

    df_text = df.iloc[idx]

    text = df_text["pn_history"]
    ents = []
    spans, feature_text, feature_num = df_text[[choice, "feature_text", "feature_num"]].values
    for s in spans:
        ents.append({"start": int(s[0]), "end": int(s[1]), "label": feature_text})

    options["colors"][feature_text] =  f"rgb{tuple(np.random.randint(100, 255, size=3))}"

    doc = {"text": text, "ents": sorted(ents, key=lambda i: i["start"])}

    spacy.displacy.render(doc, style="ent", options=options, manual=True, jupyter=True)

# **MODEL PREDICT HAS LOW F1 (<0.5)**

In [ ]:
print_error("low",10)

# **MODEL PREDICT HAS MEDIUM F1 (0.5 <= f1 < 0.7)**

In [ ]:
print_error("medium",10)

# **MODEL PREDICT HAS HIGH F1 (0.7 <= f1 < 1.0)**

In [ ]:
print_error("high",10)

In [ ]:
def plot_annotation(df, pn_num, choice="span"):
    options = {"colors": {}}

    df_text = df[df["pn_num"] == pn_num].reset_index(drop=True)

    text = df_text["pn_history"][0]
    ents = []

    for spans, feature_text, feature_num in df_text[[choice, "feature_text", "feature_num"]].values:
        for s in spans:
            ents.append({"start": int(s[0]), "end": int(s[1]), "label": feature_text})

        options["colors"][feature_text] =  f"rgb{tuple(np.random.randint(100, 255, size=3))}"

    doc = {"text": text, "ents": sorted(ents, key=lambda i: i["start"])}

    spacy.displacy.render(doc, style="ent", options=options, manual=True, jupyter=True)